In [16]:
#load libraries 
import pandas as pd
import altair as alt
import numpy as np

In [18]:
stadium = pd.read_csv("clean_stadium.csv")
fanbase = pd.read_csv("clean_fanbase.csv")
merch = pd.read_csv("clean_merch.csv")

In [26]:
merged_spending = merch.merge(
    fanbase[['Membership_ID', 'Age_Group', 'Seasonal_Pass']], 
    left_on='Member_ID', 
    right_on='Membership_ID', 
    how='left'
)

# Use cleaner fanbase age group
merged_spending['Final_Age_Group'] = merged_spending['Age_Group'].fillna(
    merged_spending['Customer_Age_Group']
)

# calc total spending per age group and seasonal pass
age_pass_spending = (
    merged_spending.groupby(['Final_Age_Group', 'Seasonal_Pass'], as_index=False)['Unit_Price']
    .sum()
    .rename(columns={'Unit_Price': 'Total_Spending', 'Final_Age_Group': 'Age_Group'})
)

# use clean labels
age_pass_spending['Seasonal_Pass'] = age_pass_spending['Seasonal_Pass'].fillna(False)
age_pass_spending['Pass_Status'] = age_pass_spending['Seasonal_Pass'].map({
    True: 'Season Pass',
    False: 'No Season Pass'
})

age_order = ['<18', '18-25', '26-40', '41-60', '60+']

chart = (
    alt.Chart(age_pass_spending)
    .mark_bar(size=40)
    .encode(
        x=alt.X('Age_Group:N', title='Age Group', sort=age_order),
        y=alt.Y('Total_Spending:Q', title='Total Merchandise Spending ($)'),
        color=alt.Color('Pass_Status:N', title='Season Pass Status', scale=alt.Scale(scheme='category10')),
        column=alt.Column('Pass_Status:N', title='Season Pass Status'), 
        tooltip=[
            alt.Tooltip('Age_Group:N', title='Age Group'),
            alt.Tooltip('Pass_Status:N', title='Season Pass'),
            alt.Tooltip('Total_Spending:Q', title='Total Spending', format='$,.0f')
        ]
    )
    .properties(
        title='Merchandise Spending by Age Group and Season Pass Status',
        width=250,
        height=300
    )
    .configure_axis(labelFontSize=12, titleFontSize=13)
    .configure_title(fontSize=16, fontWeight='bold')
)

chart


alt.Chart(...)